In [ ]:
from random import random
from random import seed
from math import exp

In [ ]:

#******************************** Intialize NN ************************

def initialize_network(n_inputs, n_hidden, n_outputs):
  NN = list()
  hidden_layer = list()# [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
  for i in range(n_hidden):
    hidden = list()
    for j in range(n_inputs+1):
      hidden.append(random())
    hidden_layer.append(hidden)
  output_layer = list() #[{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
  for i in range(n_outputs):
    output = list()
    for j in range(n_hidden+1):
      output.append(random())
    output_layer.append(output)
  NN.append(hidden_layer)
  NN.append(output_layer)
  return NN
 
#******************************** Forward Propagation **************************
#Forward Propagation: truyền 1 tín hiệu đầu vào, cho xuyên suốt toàn bộ các layer cho tới khi ra được output
#Sử dụng nhằm predict new data
#Chúng ta chia thành 3 phần
 
#1. Neuron activation (hàm kích hoạt neuron)- (nhưng dành cho 1 neuron, chỉ là tổng tích weights và input của neuron đó)
def activate(weights, inputs):
  activation = weights[-1]*1 #bias, mặc định có input = 1 và có thứ tự ở cuối weights
  for i in range(len(weights)-1):
    activation += weights[i]*inputs[i] #cộng tất cả các tham số đầu vào (weight_i*input_i)
  return activation #return về 1 số
 
#2. Neuron transfer (hay còn gọi là activation function) đây chính là lúc đưa output đã activate thành output chuẩn dạng (Eg: sgn, sigmoid...)
 
def transfer(activation):
  return 1.0/(1.0+exp(-activation)) #Sigmoid
 
#3. Forward Propagation
#forward propagation là quá trình các tính toán của mỗi layer, bao gồm việc activate các neuron, transfer các kết quả vừa ra và đưa row output vừa xong thành input của layer tiếp theo
 
def forward_propagate(network, row):
  inputs = row
  for layer in network:
    new_inputs = list()
    # print(layer,"kkk")
    for neurons in layer:
      #print(neurons)
      activated = activate(neurons, inputs)
      new_neuron = transfer(activated)
      new_inputs.append(new_neuron)
    inputs = new_inputs
  return inputs
 
#thử demo 2 input value(2 neuron), 1 hidden layer(1 neuron) và 1 output(2 neurons)
#2 input là 0 vs 1, hidden có 2 neuron và 1 bias
#5 dòng dưới đây là test forward
"""network = [[[0.13436424411240122, 0.8474337369372327, 0.763774618976614]],
        [[0.2550690257394217, 0.49543508709194095],[0.4494910647887381, 0.651592972722763]]]
row = [1, 0, None]
output = forward_propagate(network, row)
print(output)"""
#*********************************Backpropagation**********************************
#Backprogation dùng nhằm train các weights trong NN
#Chúng ta chia làm 2 phần:
#1. Transfer Derivative (hay còn gọi là đạo hàm của activation function)
 
def transfer_derivative(output):
  return output*(1.0-output) #đạo hàm của sigmoid là x*(1-x)
 
#2. Error Backpropagation
# Bước đầu là tính toán error cho lớp neuron output, từ đó sẽ cho chúng ta biết neuron nào bị sai lệch (error) nhiều để lan truyền (propagation) ngược trong network
# Công thức tính error: error = (expected - output) * transfer_derivative(output) (1)
# Với expected là gt mong đợi, output là gt thực tế.
# Tuy nhiên, với các lớp hidden, công thức sẽ có chút khác biệt
# Công thức tính error: error = (weight_k * error_j) * transfer_derivative(output) (2)
# Với weight_k là weight mà kết nối từ neuron thứ k của layer trước với neuron hiện tại, error_j là error của neuron thứ j của layer trước và output là output của neuron hiện thời
 
def backward_propagate_error(network, expected): #1 network sẽ gồm nhiều layer, 1 layer sẽ gồm nhiều neuron, 1 neuron sẽ bao gồm nhiều thuộc tính như output, weight or error(delta)
# ta sẽ mặc định network[-1] là layer output, còn lại là hidden, 1 neuron sẽ có output là pt thứ 0, xong tiếp theo là dãy weight (pt thứ 1) và cuối cùng là delta errors (pt thứ 2, tuy nhiên ở layer output sẽ k có thuộc tính này)
  for i in reversed(range(len(network))):
    layer = network[i]
    errors = list()
    if i == len(network)-1: #trường hợp ở output layer
      for j in range(len(layer)):
        neuron = layer[j] #mặc định
        #print(neuron[0])
        errors.append(expected[j] - neuron[0]) #theo công thức (1)
    else: #trường hợp ở hidden layer
      for j in range(len(layer)):
        error = 0.0
        for neuron in network[i+1]:
          print(neuron)
          error += (neuron[1][j]*neuron[2]) #theo công thức 2
        errors.append(error)
    for j in range(len(layer)):
      neuron = layer[j] #lấy neuron là layer[j] chứ k phải tạo bản sao
      #print(errors[j]*transfer_derivative(neuron[0]))
      neuron.append(errors[j]*transfer_derivative(neuron[0]))
 
# network = [[[0.7105668883115941, [0.13436424411240122, 0.8474337369372327, 0.763774618976614]]],
#       [[0.6213859615555266, [0.2550690257394217, 0.49543508709194095]], [0.6573693455986976,[0.4494910647887381, 0.651592972722763]]]]
# network=[[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
#       [{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095]}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763]}]]
# expected = [0, 1]
 
# backward_propagate_error(network, expected)
# for layer in network:
#   print(layer)
 
# ********************************* Train Network ***************************************
# Sử dụng SGD, hay còn gọi là tách training set ra thành các batch, mỗi batch có size là 1.
# Quy trình: Forward inputs qua các neuron để tạo các weights, sau đó backpropagating error và cập nhật weights
# Chúng ta chia làm 2 phần:
# 1. Update weights (cập nhật weights): #công thức weight = weight + learning_rate*error*input
# trong đó: learning_rate giới hạn độ thay đổi là nhanh hay chậm, error tính từ backpropagation và input tương ứng
def update_weights(network, row, l_rate):
  eta = l_rate
  for i in range(len(network)):
    inputs = row[:-1]
    if i != 0:
      inputs = [neuron[0] for neuron in network[i-1]]
    for neuron in network[i]:
      for j in range(len(inputs)):
        neuron[1][j] += eta*neuron[2]*inputs[j]
      neuron[1][-1] += eta*neuron[2]
 
# 2. Train network: sử dụng SGD, gồm 1 số lượng cố định số lượng các epochs và cập nhật mỗi epoch theo hàng trong dataset
# expected number of output (n_outputs) được dùng để biến đổi lớp giá trị đầu vào thành vector binary (one hot coding)
def training_network(network, trainset, l_rate, n_epoch, n_outputs):
  for epoch in range(n_epoch):
    sum_error = 0
    for row in trainset:
      # print(row)
      outputs = forward_propagate(network,row)
      # print(outputs)
      expected = [0 for i in range(n_outputs)]
      expected[row[-1]] = 1
      sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
      backward_propagate_error(network, expected)
      update_weights(network, row, l_rate)
    print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
 

In [ ]:
seed(1)
dataset = [[2.7810836,2.550537003,0],
    [1.465489372,2.362125076,0],
    [3.396561688,4.400293529,0],
    [1.38807019,1.850220317,0],
    [3.06407232,3.005305973,0],
    [7.627531214,2.759262235,1],
    [5.332441248,2.088626775,1],
    [6.922596716,1.77106367,1],
    [8.675418651,-0.242068655,1],
    [7.673756466,3.508563011,1]]
n_inputs = len(dataset[0]) - 1 #số lượng feature - số lượng input đầu vào = 2 (-1 là output)
n_outputs = len(set([row[-1] for row in dataset])) #do chỉ có giá trị 0 và 1
# print(n_inputs,n_outputs)
network = initialize_network(n_inputs, 2, n_outputs) #2 input (1 layer), 2 hidden (1 layer), 2 output (1 layer)
# => tồn tại 2 neuron với hidden và 1 output với 2 neuron, mỗi neuron sẽ có 3 attribute
# vì sao lại có 3 attribute vì có 2 feature + 1 bias
# for layer in network:
#   print(layer)
# print(network)
training_network(network, dataset, 0.5, 20, n_outputs)
# for layer in network:
    # print(layer)
 
# seed(1)
# net = initialize_network(2,1,2)
# for layer in net:
# print(layer)

[0.651592972722763, 0.7887233511355132, 0.0938595867742349, 0.07909521373384375]


TypeError: ignored

In [ ]:
import numpy as np
# np.array(network).shape
print(len(network))

2


In [ ]:
from math import exp
import numpy as np
from random import seed
from random import random
 
# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network
 
# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation
 
# Transfer neuron activation
def transfer(activation):
	return 1.0 / (1.0 + exp(-activation))
 
# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs
 
# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)
 
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])
 
# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] += l_rate * neuron['delta']
 
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		sum_error = 0
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)
		print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
 
# Test training backprop algorithm
seed(1)
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 2, n_outputs)
# for layer in network:
#   print("layer")
#   for neuron in layer:
#     print("Output:",neuron['output'], "Weights:",neuron['weights'])
for layer in network:
  print(layer)
train_network(network, dataset, 0.5, 20, n_outputs)
for layer in network:
	print(layer)

(2, 2)
[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}, {'weights': [0.2550690257394217, 0.49543508709194095, 0.4494910647887381]}]
[{'weights': [0.651592972722763, 0.7887233511355132, 0.0938595867742349]}, {'weights': [0.02834747652200631, 0.8357651039198697, 0.43276706790505337]}]
>epoch=0, lrate=0.500, error=6.350
>epoch=1, lrate=0.500, error=5.531
>epoch=2, lrate=0.500, error=5.221
>epoch=3, lrate=0.500, error=4.951
>epoch=4, lrate=0.500, error=4.519
>epoch=5, lrate=0.500, error=4.173
>epoch=6, lrate=0.500, error=3.835
>epoch=7, lrate=0.500, error=3.506
>epoch=8, lrate=0.500, error=3.192
>epoch=9, lrate=0.500, error=2.898
>epoch=10, lrate=0.500, error=2.626
>epoch=11, lrate=0.500, error=2.377
>epoch=12, lrate=0.500, error=2.153
>epoch=13, lrate=0.500, error=1.953
>epoch=14, lrate=0.500, error=1.774
>epoch=15, lrate=0.500, error=1.614
>epoch=16, lrate=0.500, error=1.472
>epoch=17, lrate=0.500, error=1.346
>epoch=18, lrate=0.500, error=1.233
>epoch=19, lrat